Group Project Proposal

Introduction:

The Data Science Salaries dataset, last updated on February 27th of this year, includes metrics on jobs in the Data Science industry from 2020 to 2022. We will use it to find the relationship between salaries, demographics, education, and experience levels,  for data science jobs. Data for this set was collected through industry surveys, job surveys, recruitment agencies, and company reports. For this project, we will answer the predictive question, “Can we predict the ideal location (employee residence) to work a data science job based on the highest yearly salary in USD, experience level, and job title?” using classification. We want to answer this question to give workers insight about which location is most ideal to work a certain job in terms of salary. 


Preliminary Data Analysis:

In [35]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

In [24]:
ds_salaries <- read_csv("data/ds_salaries.csv") |>
    mutate(employee_residence = as_factor(employee_residence))

Rows: 607 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (7): experience_level, employment_type, job_title, salary_currency, empl...
dbl (5): id, work_year, salary, salary_in_usd, remote_ratio

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [25]:
slice(ds_salaries, 1:6)

id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<fct>,<dbl>,<chr>,<chr>
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
5,2020,EN,FT,Data Analyst,72000,USD,72000,US,100,US,L


In [27]:
en_data <- filter(ds_salaries, experience_level == "EN")

In [ ]:
mi_data <- filter(ds_salaries, experience_level == "MI")

In [28]:
se_data <- filter(ds_salaries, experience_level == "SE")

In [38]:
en_split <- initial_split(ds_salaries, prop = 0.75, strata = employee_residence)  
en_train <- training(en_split)   
en_test <- testing(en_split)

In [39]:
mi_split <- initial_split(ds_salaries, prop = 0.75, strata = employee_residence)  
mi_train <- training(en_split)   
mi_test <- testing(en_split)

In [36]:
se_split <- initial_split(ds_salaries, prop = 0.75, strata = employee_residence)  
se_train <- training(en_split)   
se_test <- testing(en_split)

Methods:

We will be using the data columns, experience level, monthly salary, employee residency, and job title. From this, we will filter our data by experience level (entry (EN), middle (MI), and senior (SE)) to create three comparable data sets and produce three scatterplots to perform the K-nn algorithm accordingly. 

Describe at least one way that you will visualise the results:

One way we will visualise the results is by using a bar plot to compare the average salary across different locations.he location?”


Expected outcomes and significance:

What do you expect to find?

We expect to find that the US offers the highest mean salary for data scientists due to the high number of tech successful start-ups and blue-chip companies based there, such as Apple, Amazon, and Microsoft.

What impact could such findings have?

By showing the trends of salary distribution among different locations, our findings could lead to a shift in the residing location of individuals going into the data science industry.  

What future questions could this lead to?

As a result of our data exploration, future questions could arise concerning factors beyond what is included in the variables of this data set, such as, “How is salary based on cost of living?”, or “How does access to education and resources affect salary?”, as well as more questions based on what is available in the dataset, such as “Is salary for the same job dependent on the location?”